In [1]:
import subprocess
from pathlib import Path
import shutil
import pandas as pd

In [18]:
sig = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/results/DE_featureCounts/Tanner/P5-OE_P5-WT/Tanner_Mouse_P5-OE_P5-WT_sig.csv", sep=',', index_col=0)
sig

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSMUSG00000067879.4,10.829698,7.223096,2.103328,3.434127,0.000594,0.031250
ENSMUSG00000026110.16,122.058345,-2.819427,0.835169,-3.375877,0.000736,0.035101
ENSMUSG00000026090.17,60.991841,-5.156713,1.563390,-3.298417,0.000972,0.042806
ENSMUSG00000094497.2,1573.195377,-0.338548,0.104645,-3.235190,0.001216,0.049553
ENSMUSG00000026043.19,1282.232905,-1.523763,0.314251,-4.848873,0.000001,0.000337
...,...,...,...,...,...,...
ENSMUSG00000050332.8,453.190903,-1.075014,0.252659,-4.254806,0.000021,0.002858
ENSMUSG00000083327.2,454.504351,2.367331,0.552360,4.285852,0.000018,0.002552
ENSMUSG00000079316.11,2045.824484,-0.497444,0.141674,-3.511178,0.000446,0.026087
ENSMUSG00000072844.7,80.944792,2.427800,0.742633,3.269178,0.001079,0.045966


In [20]:
gct = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/Tanner.gct",sep='\t')
gct = gct[gct['Description'].isin(sig.index)]
gct.to_csv("Tanner_sig.gct",sep='\t',index=False)
gct

,Name,Description,21415-P5-OE-3,31284-P5-OE-2,31351-P5-OE-1,31391-P5-WT-1,31395-P5-WT-3,31396-P5-WT-2
107,ENSMUSG00000067879,ENSMUSG00000067879.4,0.000000,3.822675,82.190846,0.000000,0.000000,0.000000
552,ENSMUSG00000026110,ENSMUSG00000026110.16,40.892133,30.581402,61.141971,67.875196,766.748522,99.500156
555,ENSMUSG00000026090,ENSMUSG00000026090.17,8.178427,1.911338,8.018619,3.572379,615.813774,24.875039
656,ENSMUSG00000094497,ENSMUSG00000094497.2,1519.369919,1379.030103,1348.130342,1681.697283,1819.669327,1868.737310
698,ENSMUSG00000026043,ENSMUSG00000026043.19,624.286563,506.504473,781.815365,1286.056340,2347.337208,1859.409170
...,...,...,...,...,...,...,...,...
54098,ENSMUSG00000050332,ENSMUSG00000050332.8,246.261512,318.237716,331.770367,660.890063,687.054975,538.959180
54307,ENSMUSG00000083327,ENSMUSG00000083327.2,321.684779,409.026254,2620.083802,217.915102,205.271258,223.875352
55239,ENSMUSG00000079316,ENSMUSG00000079316.11,2061.872216,1657.129729,1603.723827,2793.600160,2303.868000,2414.951709
55285,ENSMUSG00000072844,ENSMUSG00000072844.7,19.991709,41.093759,76.176882,3.572379,9.659824,12.437520


In [65]:
with open("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/mouse/meta_mouse.txt", "r") as f:
    files = [i.strip() for i in f.readlines()]
for pair in files:
    g1,g1_f,g2,g2_f = pair.split()
    print(g1, g2)
    name = g1+"_"+g2

    log_dir = Path(
        f"/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/log/vast_out")
    log_dir.mkdir(parents=True, exist_ok=True)

    path_lsf = Path(
        f"/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/lsf/vast_out")
    path_lsf.mkdir(parents=True, exist_ok=True)

    out_dir = Path(f"new/vast_out/mouse/{name}")
    to_combine = out_dir.joinpath("to_combine")
    to_combine.mkdir(parents=True, exist_ok=True)
    g1_files = [i for i in list(Path(f"new/vast_out/mouse/to_combine").glob("*")) if any([j for j in g1_f.split(",") if j in str(i)])]
    if g2=="WT":
        g2_files = [i for i in list(Path(f"new/vast_out/mouse/to_combine").glob("*")) if (any([j for j in g2_f.split(",") if j in str(i)])) and ("P5" not in str(i))]
    else:
        g2_files = [i for i in list(Path(f"new/vast_out/mouse/to_combine").glob("*")) if any([j for j in g2_f.split(",") if j in str(i)])]
    assert len(g1_files)==21, print(len(g1_files))
    assert len(g2_files)==21, print(len(g2_files))
    [shutil.copy(file, to_combine.joinpath(file.name)) for file in g1_files]
    [shutil.copy(file, to_combine.joinpath(file.name)) for file in g2_files]
    
    command = f"#BSUB -W 6:00\n" \
        f"#BSUB -q medium\n" \
        f"#BSUB -n 8\n" \
        f"#BSUB -M 100\n" \
        f"#BSUB -R rusage[mem=100]\n" \
        "#BSUB –u nkdang@mdanderson.org\n" \
        f"#BSUB -J vast_{name}\n" \
        f"#BSUB -o {log_dir.joinpath(name)}.log\n" \
        f"#BSUB –cwd /rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects\n" \
        f"./vast-tools/vast-tools combine -sp mm10 --cores 12 -o {out_dir}\n" \
        f"./vast-tools/vast-tools compare {out_dir}/INCLUSION_LEVELS_FULL-mm10-6.tab -a {g1_f} -b {g2_f} -name_A {g1} -name_B {g2} --outFile vast_compare_{name}.tab\n" \
        f"./vast-tools/vast-tools diff -a {g1_f} -b {g2_f} -o {out_dir} -d vast_diff_{name} -c 12\n" \

    with open(path_lsf.joinpath(f'{name}.lsf'), "w+") as out:
        out.writelines(command)

    command = f"bsub < {path_lsf.joinpath(f'{name}.lsf')}"
    subprocess.Popen(command, shell=True)


KI WT
KO WT
Job <23761778> is submitted to queue <medium>.
KI KO
Job <23761779> is submitted to queue <medium>.
P5-OE P5-WT


Job <23761780> is submitted to queue <medium>.
Job <23761781> is submitted to queue <medium>.


In [62]:
with open("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/meta_human.txt", "r") as f:
    files = [i.strip() for i in f.readlines()]

for pair in files:
    g1, g1_f, g2, g2_f = pair.split()
    print(g1, g2)
    name = g1 + "_" + g2

    log_dir = Path(
        f"/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/log/vast_out")
    log_dir.mkdir(parents=True, exist_ok=True)

    path_lsf = Path(
        f"/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/lsf/vast_out")
    path_lsf.mkdir(parents=True, exist_ok=True)

    out_dir = Path(f"new/vast_out/human/{name}")
    to_combine = out_dir.joinpath("to_combine")
    to_combine.mkdir(parents=True, exist_ok=True)
    g1_files = [i for i in list(Path(f"new/vast_out/human/to_combine").glob("*")) if any([j for j in g1_f.split(",") if j in str(i)])]
    g2_files = [i for i in list(Path(f"new/vast_out/human/to_combine").glob("*")) if any([j for j in g2_f.split(",") if j in str(i)])]
    assert len(g1_files)==21, print(len(g1_files))
    assert len(g2_files)==21, print(len(g2_files))
    # [shutil.copy(file, to_combine.joinpath(file.name)) for file in g1_files]
    # [shutil.copy(file, to_combine.joinpath(file.name)) for file in g2_files]

    command = f"#BSUB -W 6:00\n" \
        f"#BSUB -q medium\n" \
        f"#BSUB -n 8\n" \
        f"#BSUB -M 100\n" \
        f"#BSUB -R rusage[mem=100]\n" \
        "#BSUB –u nkdang@mdanderson.org\n" \
        f"#BSUB -J vast_{name}\n" \
        f"#BSUB -o {log_dir.joinpath(name)}.log\n" \
        f"#BSUB –cwd /rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects\n" \
        f"./vast-tools/vast-tools diff -a {g1_f} -b {g2_f} --sampleNameA {g1} --sampleNameB {g2} -o {out_dir} -d vast_diff_{name} -c 12 -n 1000\n" \
        # f"./vast-tools/vast-tools combine -sp hg38 --cores 12 -o {out_dir}\n" \
        # f"./vast-tools/vast-tools compare {out_dir}/INCLUSION_LEVELS_FULL-hg38-6.tab -a {g1_f} -b {g2_f} -name_A {g1} -name_B {g2} --outFile vast_compare_{name}.tab\n" \

    with open(path_lsf.joinpath(f'{name}.lsf'), "w+") as out:
        out.writelines(command)

    command = f"bsub < {path_lsf.joinpath(f'{name}.lsf')}"
    subprocess.Popen(command, shell=True)


RFH838-DMSOR-DMSO RFH838-DMSOR-MRTX
RFH838-MRTXR-DMSO RFH838-MRXTR-MRTX
RFH838-DMSOR-DMSO RFH838-MRTXR-DMSO
RFH838-DMSOR-MRTX RFH838-MRXTR-MRTX
Job <23752674> is submitted to queue <medium>.
Job <23752675> is submitted to queue <medium>.
Job <23752676> is submitted to queue <medium>.
RFH1437-DMSOR-DMSO RFH1437-DMSOR-MRTX
Job <23752677> is submitted to queue <medium>.
RFH1437-MRTXR-DMSO RFH1437-MRXTR-MRTX
Job <23752678> is submitted to queue <medium>.
RFH1437-DMSOR-DMSO RFH1437-MRTXR-DMSO
Job <23752679> is submitted to queue <medium>.
RFH1437-DMSOR-MRTX RFH1437-MRXTR-MRTX
Job <23752680> is submitted to queue <medium>.
Hela_DMSO Hela_EPZ
Job <23752681> is submitted to queue <medium>.
Hela_EV Hela_WT
Job <23752682> is submitted to queue <medium>.
Hela_Y112A Hela_WT
Job <23752683> is submitted to queue <medium>.
Hela_W377A Hela_WT
Job <23752684> is submitted to queue <medium>.
Hela_Y112A Hela_EV
Job <23752685> is submitted to queue <medium>.
Hela_W377A Hela_EV
Job <23752686> is submitted t

Job <23752688> is submitted to queue <medium>.


In [90]:
for f in Path("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/").glob("**/vast_diff*.tab"):
    if "_sig.tab" in str(f):
        continue
    print(f)
    vast_diff = pd.read_csv(f, sep='\t')
    out = f.parent.joinpath(f.stem+"_sig.tab")
    vast_diff[(vast_diff['MV[dPsi]_at_0.95']!=0) & (vast_diff['E[dPsi]'].abs()>0.005)].reset_index(drop=True).to_csv(out, index=False, sep='\t')

/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/RFH838-DMSOR-DMSO_RFH838-DMSOR-MRTX/vast_diff_RFH838-DMSOR-DMSO_RFH838-DMSOR-MRTX.tab
/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/Hela_W377A_Hela_WT/vast_diff_Hela_W377A_Hela_WT.tab
/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/Hela_Y112A_Hela_EV/vast_diff_Hela_Y112A_Hela_EV.tab
/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/RFH838-DMSOR-MRTX_RFH838-MRXTR-MRTX/vast_diff_RFH838-DMSOR-MRTX_RFH838-MRXTR-MRTX.tab
/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/Hela_Y112A_Hela_WT/vast_diff_Hela_Y112A_Hela_WT.tab
/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/RFH838-DMSOR-DMSO_RFH838-MRTXR-DMSO/vast_diff_RFH838-DMSOR-DMSO_RFH838-MRTXR-DMSO.tab
/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_o

In [3]:
# vast_compare = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/Hela_DMSO_Hela_EPZ/vast_compare_Hela_DMSO_Hela_EPZ.tab", sep='\t')
vast_diff = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/Hela_DMSO_Hela_EPZ/vast_diff_Hela_DMSO_Hela_EPZ_sig.tab", sep='\t')
inclusion_full = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/vast_out/human/Hela_DMSO_Hela_EPZ/INCLUSION_LEVELS_FULL-hg38-6.tab", sep='\t')
rMATS = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/rMATS/human/Yalong_WT-DMSO_WT-EPZ/summary.txt", sep='\t', index_col=0)
inclusion_full

,GENE,EVENT,COORD,LENGTH,FullCO,COMPLEX,Hela-WT_DMSO_4_day__Replicate_1,Hela-WT_DMSO_4_day__Replicate_1-Q,Hela-WT_DMSO_4_day__Replicate_2,Hela-WT_DMSO_4_day__Replicate_2-Q,Hela-WT_DMSO_4_day__Replicate_3,Hela-WT_DMSO_4_day__Replicate_3-Q,Hela-WT_EPZ_4_day__Replicate_1,Hela-WT_EPZ_4_day__Replicate_1-Q,Hela-WT_EPZ_4_day__Replicate_2,Hela-WT_EPZ_4_day__Replicate_2-Q,Hela-WT_EPZ_4_day__Replicate_3,Hela-WT_EPZ_4_day__Replicate_3-Q
0,TSPAN6,HsaEX0067681,chrX:100636191-100636207,17,"chrX:100636608+100636793,100636191-100636207,1...",S,0.0,"SOK,SOK,0=0=133,Bn,S@0.00,133.00",0.00,"SOK,SOK,0=0=120,Bn,S@0.00,120.00",4.86,"SOK,SOK,1=3=137,B1,S@6.85,134.15",0.00,"SOK,SOK,0=0=117,Bn,S@0.00,117.00",3.08,"SOK,SOK,1=1=110,OK,S@3.45,108.55",1.53,"SOK,SOK,0=1=113,Bn,S@1.74,112.26"
1,TNMD,HsaEX0066298,chrX:100599016-100599182,167,"chrX:100597657,100599016-100599182,100599508",S,NaN,"N,N,0=0=0,Bn,S@0,0",NaN,"N,N,0=0=0,Bn,S@0,0",NaN,"N,N,0=0=0,Bn,S@0,0",100.00,"N,N,0=1=0,Bn,S@1.00,0.00",NaN,"N,N,0=0=0,Bn,S@0,0",NaN,"N,N,0=0=0,Bn,S@0,0"
2,DPM1,HsaEX6078702,chr20:50955186-50955285,100,"chr20:50958363,50955186-50955285,50948662",S,100.0,"SOK,SOK,364=482=0,OK,S@846.00,0.00",100.00,"SOK,SOK,365=531=0,OK,S@896.00,0.00",100.00,"SOK,SOK,347=336=0,OK,S@683.00,0.00",100.00,"SOK,SOK,438=526=0,OK,S@964.00,0.00",100.00,"SOK,SOK,350=317=0,OK,S@667.00,0.00",100.00,"SOK,SOK,388=415=0,OK,S@803.00,0.00"
3,SCYL3,HsaEX0056692,chr1:169855796-169855857,62,"chr1:169859041,169855796-169855857+169855932+1...",S,12.5,"OK,OK,2=4=21,OK,S@3.38,23.62",6.67,"OK,OK,0=4=28,Bn,S@2.13,29.87",16.13,"VLOW,VLOW,1=4=13,B1,S@2.90,15.10",5.77,"OK,OK,8=0=66,Bn,S@4.27,69.73",0.00,"OK,OK,0=0=43,Bn,S@0.00,43.00",8.86,"OK,OK,5=1=31,B2,S@3.28,33.72"
4,SCYL3,HsaEX0056690,chr1:169864458-169864508,51,"chr1:169866896,169864369+169864458-169864508,1...",S,100.0,"OK,OK,18=37=0,B1,S@55.00,0.00",87.50,"VLOW,VLOW,19=9=2,B1,S@26.25,3.75",100.00,"VLOW,VLOW,8=22=0,B1,S@30.00,0.00",100.00,"OK,OK,17=27=0,OK,S@44.00,0.00",100.00,"LOW,LOW,21=13=0,OK,S@34.00,0.00",96.15,"OK,OK,22=28=1,OK,S@49.04,1.96"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721546,KDM5D,HsaINT0085241,chrY:19708072-19708243,172,chrY:19708244-19708423=19707934-19708071:-,IR,NaN,"N,N,0.0=123,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=123,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=123,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=123,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=123,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=123,0=0=0,1@NaN,NaN"
721547,KDM5D,HsaINT0085242,chrY:19707747-19707933,187,chrY:19707934-19708071=19707147-19707746:-,IR,0.0,"OK,OK,0.0=138,0=0=96,1@0,96",2.36,"OK,OK,0.0=138,3=0=62,0.364@1.53,63.47",0.00,"OK,OK,0.0=138,0=0=45,1@0,45",0.00,"SOK,SOK,0.0=138,0=0=132,1@0,132",0.00,"SOK,SOK,0.0=138,0=0=139,1@0,139",0.00,"OK,OK,0.0=138,0=0=87,1@0,87"
721548,KDM5D,HsaINT0085243,chrY:19706864-19707146,283,chrY:19707147-19707746=19706794-19706863:-,IR,NaN,"N,N,0.0=151,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=151,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=151,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=151,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=151,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=151,0=0=0,1@NaN,NaN"
721549,KDM5D,HsaINT0085244,chrY:19706641-19706793,153,chrY:19706794-19706863=19706441-19706640:-,IR,NaN,"N,N,0.0=104,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=104,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=104,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=104,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=104,0=0=0,1@NaN,NaN",NaN,"N,N,0.0=104,0=0=0,1@NaN,NaN"


In [4]:
inclusion_full['COMPLEX'].value_counts()

IR      192965
Alt3    171733
Alt5    133288
S       103307
ANN      55566
C3       22445
C1       21535
C2       18700
MIC       1995
Name: COMPLEX, dtype: int64

In [10]:
rMATS

,TotalEventsJC,TotalEventsJCEC,SignificantEventsJC,SigEventsJCSample1HigherInclusion,SigEventsJCSample2HigherInclusion,SignificantEventsJCEC,SigEventsJCECSample1HigherInclusion,SigEventsJCECSample2HigherInclusion
EventType,,,,,,,,
SE,49531,49921,6969,4824,2145,7173,4910,2263
A5SS,5347,5388,1053,455,598,1137,492,645
A3SS,8280,8313,1240,599,641,1288,616,672
MXE,6641,6703,924,397,527,956,395,561
RI,6522,6660,1829,515,1314,1975,567,1408


In [11]:
event_count = inclusion_full[inclusion_full['EVENT'].isin(vast_diff['EVENT'])]['COMPLEX']
event_count = event_count.replace("S","SE").replace("C1","SE").replace("C2","SE").replace("C3","SE").replace("IR","RI").replace("Alt5","A5SS").replace("Alt3","A3SS").replace("IR","RI")
event_count = pd.DataFrame(event_count[event_count.isin(['RI',"SE","A5SS","A3SS"])].value_counts())
event_count.columns = ['VAST']

results = rMATS[['SignificantEventsJC','SignificantEventsJCEC']].merge(event_count, how='left', right_index=True, left_index=True).fillna(0).astype(int)
results

,SignificantEventsJC,SignificantEventsJCEC,VAST
EventType,,,
SE,6969,7173,2772
A5SS,1053,1137,1622
A3SS,1240,1288,1323
MXE,924,956,0
RI,1829,1975,3158


In [14]:
rMATS_SE = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/rMATS/human/Yalong_WT-DMSO_WT-EPZ/SE.MATS.JCEC.txt", sep='\t', index_col=0)
rMATS_SE

,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE,...,SJC_SAMPLE_1,IJC_SAMPLE_2,SJC_SAMPLE_2,IncFormLen,SkipFormLen,PValue,FDR,IncLevel1,IncLevel2,IncLevelDifference
ID,,,,,,,,,,,,,,,,,,,,,
80,ENSG00000214827.11,MTCP1,chrX,-,155065905,155066057,155065618,155065789,155070693,155071136,...,"17,7,18","40,27,24","1,4,10",301,149,1.967204e-12,3.623793e-10,"0.08,0.175,0.076","0.952,0.77,0.543",-0.645
84,ENSG00000185010.15,F8,chrX,-,154999478,154999600,154996972,154997095,155022409,155022496,...,"0,1,0","4,0,0","0,0,0",271,149,1.000000e+00,1.000000e+00,"NA,0.0,1.0","1.0,NA,NA",-0.500
88,ENSG00000130826.19,DKC1,chrX,+,154766996,154767061,154766215,154766400,154767255,154767382,...,"15,21,12","869,530,610","3,8,8",214,149,3.441339e-04,4.300253e-03,"0.968,0.96,0.973","0.995,0.979,0.982",-0.018
92,ENSG00000130826.19,DKC1,chrX,+,154769166,154769310,154767255,154767382,154770758,154770879,...,"0,0,0","745,415,419","2,0,0",293,149,1.000000e+00,1.000000e+00,"1.0,1.0,1.0","0.995,1.0,1.0",0.002
93,ENSG00000130826.19,DKC1,chrX,+,154770758,154770879,154769046,154769310,154773130,154773249,...,"3,3,0","1262,789,765","13,0,1",270,149,8.773477e-01,1.000000e+00,"0.994,0.994,1.0","0.982,1.0,0.998",0.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84914,ENSG00000165807.8,PPP1R36,chr14,+,64552813,64552861,64550920,64550985,64564750,64564837,...,"0,4,0","10,0,3","0,0,0",197,149,1.000000e+00,1.000000e+00,"1.0,0.763,1.0","1.0,NA,1.0",-0.079
84916,ENSG00000165807.8,PPP1R36,chr14,+,64565356,64565454,64564750,64564837,64565625,64565692,...,"0,0,1","6,0,17","0,0,0",247,149,1.000000e+00,1.000000e+00,"1.0,1.0,0.916","1.0,NA,1.0",-0.028
84917,ENSG00000165807.8,PPP1R36,chr14,+,64574454,64574589,64568348,64568447,64586836,64586879,...,"2,0,3","9,7,1","0,0,0",284,149,6.063696e-05,1.050332e-03,"0.0,1.0,0.259","1.0,1.0,1.0",-0.580
